In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

base_model_name = "GAI-LLM/Yi-Ko-6B-mixed-v15"
adapter_model_name = "../model/GAI-LLM-Yi-Ko-6B-mixed-v15-sft-qlora-v1"

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.04s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or tr

In [20]:
data = pd.read_csv('../data/eval_v1.csv')

In [21]:
data = data.sample(n=2000)

In [22]:
data.head(2)

,Unnamed: 0,질문,답변
6212,136863,비닐장판의 기대수명은 얼마야?장판 견적을 계산하는 데 중요한 요소는 무엇인가요?,"비닐장판의 기대수명은 10년입니다.장판 견적은 시공현장의 상태, 장판 소재, 면적 ..."
10601,112415,인테리어에 천장 장식품을 사용하는 방법은 무엇인가요?벽에 구멍을 막는데 사용할 수 ...,천장에 모델링이나 컬러풀한 조명을 활용하여 공간에 독특하고 아름다운 분위기를 조성할...


In [23]:
len(data)

2000

In [24]:
def extract_text(input_string):
    index_t = input_string.find('<|assistant|>')
    if index_t != -1:  
        result = input_string[index_t + len('<|assistant|>'):]
    else: 
        raise Exception
    return result

In [ ]:
# 2000 x 10
questions = []
inference_results = []
PROMPT_TEMPLATE = '<|user|>{question}{sep_token}<|assistant|>'

for i in tqdm(range(len(data))):
    result = []
    row = data.iloc[i]

    question = row['질문']
    answer = row['답변']

    prompt = PROMPT_TEMPLATE.format(question=question,
                                     sep_token=tokenizer.eos_token)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, 
                         max_length=512, 
                         num_beams=10,
                         repetition_penalty=1.5,
                         diversity_penalty=0.5,
                         num_beam_groups=5,
                         num_return_sequences=10)
    
    for k in range(10):
        response = tokenizer.decode(outputs[k], skip_special_tokens=True)
        response = extract_text(response)
        result.append(response)
    
    questions.append(question)
    inference_results.append(result)

In [30]:
len(questions), len(inference_results)

(1453, 1453)

In [31]:
# 모든 결과 저장
sft_inf_results = {'data':[]}
for q, result in zip(questions,inference_results):
    item = {
        'question':q,
        'result':result
    }
    sft_inf_results['data'].append(item)

import json
with open("../data/dpo/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1-dpo-raw.json", "w") as json_file:
    json.dump(sft_inf_results, json_file, ensure_ascii=False, indent=4)


# embedding 비교

In [35]:
from sentence_transformers import SentenceTransformer 

embed_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
embed_model =  embed_model.to(device)

In [36]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [37]:
prompt = []
chosen = []
rejected = []

for i in tqdm(range(len(inference_results))):
    row = data.iloc[i]
    preds = inference_results[i]

    answer = row['답변']

    gt_embed = embed_model.encode(answer)
    sample_score_list = []

    for pred in preds:
        pred_embed = embed_model.encode(pred)
        sample_score = cosine_similarity(gt_embed, pred_embed)
        sample_score_list.append(sample_score)
    
    max_idx = sample_score_list.index(max(sample_score_list))
    min_idx = sample_score_list.index(min(sample_score_list))
    
    prompt.append(row['질문'])
    chosen.append(preds[max_idx])
    rejected.append(preds[min_idx])


100%|██████████| 1453/1453 [01:28<00:00, 16.41it/s]


In [40]:
dpo_df = pd.DataFrame({
        'prompt':prompt,
        'chosen':chosen,
        'rejected':rejected
    }
)
dpo_df.head()

,prompt,chosen,rejected
0,비닐장판의 기대수명은 얼마야?장판 견적을 계산하는 데 중요한 요소는 무엇인가요?,"비닐장판의 기대수명은 10년입니다.장판 견적은 시공현장의 상태, 장판 소재, 면적 ...","비닐장판은 일반적으로 10년 정도의 수명을 가지고 있습니다. 그러나 사용 환경, 방..."
1,인테리어에 천장 장식품을 사용하는 방법은 무엇인가요?벽에 구멍을 막는데 사용할 수 ...,천장 장식품을 사용하는 방법은 다양합니다. 먼저 모델링을 사용하여 천장에 장식을 부...,인테리어에 천장 장식품을 사용하는 방법은 다양합니다. 천장에 모델링이나 컬러풀한 조...
2,리얼징크의 단점은 뭐야?인테리어에서 창문 커버링을 선택할 때 고려해야 하는 것들은 ...,"리얼징크의 가장 큰 단점은 다른 종류의 징크에 비해 수명이 짧다는 것입니다. 또한,...",시공이 부실하면 오염이 쉽게 발생할 수 있습니다.리얼징크의 가격이 다른 소재에 비해...
3,배관공사에 사용되는 소제구가 뭐야?벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,배관공사에 사용되는 소제구란 배수관이 막혔을때 뚫기위해 설치하는 구멍을 의미합니다....,배관공사의 일부로 배관 내부의 이물질을 제거하기 위해 사용하는 도구입니다. 이 도구...
4,발포우레탄폼의 장점이 뭐야?실내 조명을 선택할 때 어떤 종류의 조명을 고려해야 하나요?,발포우레탄폼은 방수에 효과적이고 단열성능이 좋다는 장점이 있습니다.실내 조명을 선택...,배관공사 시에 사용되는 소제구는 배수관이 막힐 경우 점검과 청소를 위해 설치하는 구...


In [43]:
from sklearn.model_selection import train_test_split
train_ratio = 0.9 
train_df, dev_df = train_test_split(dpo_df, test_size=1-train_ratio, random_state=42)
print(train_df.shape)

(1307, 3)


In [44]:
train_df.to_csv('../data/dpo/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1-dpo-train.csv')
dev_df.to_csv('../data/dpo/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1-dpo-eval.csv')